In [1]:
%pip install pydub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import socket
import numpy as np
import scipy
import io
from pydub import AudioSegment

In [3]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('', 1234))

In [11]:
wav_test = []

while True:
  message, address = server_socket.recvfrom(1024)
  data = list(message)
  assert len(data) == 1024
  wav_test.extend(data)

KeyboardInterrupt: 

In [12]:
len(wav_test)

172032

In [13]:
s = io.BytesIO(bytes(wav_test))
audio = AudioSegment.from_raw(s, sample_width=2, frame_rate=16000, channels=1).export('test-02.wav', format='wav')